In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import os
import random
from PIL import Image

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [3]:
img_shape = np.array([28,28])
num_input = img_shape[0] * img_shape[1]
num_classes = 10

drop_out = 0.2
learning_rate = 0.001
epochs = 5
batch_size = 128
display_step = float("inf")

In [4]:
X_TF = tf.placeholder(tf.float32, [None, num_input])
Y_TF = tf.placeholder(tf.float32, [None, num_classes])
DROP_OUT_TF = tf.placeholder(tf.float32)

In [5]:
def conv_net(X_TF, DROP_OUT):
    X_TF = tf.reshape(X_TF, shape=[-1, img_shape[0], img_shape[1], 1])
    
    conv1 = tf.layers.conv2d(
        inputs=X_TF,
        filters=32,
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)
    
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],strides=2)
    
    pool2_flat = tf.reshape(pool2,[-1,7*7*64])
    dense = tf.layers.dense(inputs=pool2_flat,units=1024,activation=tf.nn.relu)
    dropout = tf.layers.dropout(
        inputs=dense,
        rate=DROP_OUT)

    return tf.layers.dense(inputs=dropout,units=num_classes)

In [6]:
logits = conv_net(X_TF,DROP_OUT_TF)
prediction = tf.nn.softmax(logits)

cost = tf.reduce_mean(-tf.reduce_sum(Y_TF * tf.log(prediction), reduction_indices=[1]))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y_TF, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        num_steps = int(len(mnist.train.labels)/batch_size)
        for step in range(num_steps):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={X_TF: batch_x, Y_TF: batch_y, DROP_OUT_TF:drop_out})
            if step % display_step == 0:
                print("step:",step,"- epoch:",epoch)
                test_pred = sess.run(prediction,feed_dict={X_TF: mnist.test.images[:5]})
                print("y_hat:",np.argmax(test_pred,axis=1))
                print("y    :",np.argmax(mnist.test.labels[:5],axis=1))
                print("train_cost:",sess.run(cost,feed_dict={X_TF:mnist.train.images[:1000],Y_TF:mnist.train.labels[:1000],DROP_OUT_TF:0}))
                print("valid_cost:",sess.run(cost,feed_dict={X_TF:mnist.validation.images,Y_TF:mnist.validation.labels,DROP_OUT_TF:0}))
                print("Accuracy:",sess.run(accuracy, feed_dict={X_TF:mnist.validation.images,Y_TF:mnist.validation.labels,DROP_OUT_TF:0}))
                print("________________________________________")

    print()
    print()
    print("Optimization Finished!")
    print("Test accuracy:",sess.run(accuracy, feed_dict={X_TF:mnist.test.images,Y_TF:mnist.test.labels,DROP_OUT_TF:0}))

step: 0 - epoch: 0
y_hat: [9 9 9 9 9]
y    : [7 2 1 0 4]
train_cost: 2.1813705
valid_cost: 2.1981673
Accuracy: 0.1726
________________________________________
step: 0 - epoch: 1
y_hat: [7 2 1 0 4]
y    : [7 2 1 0 4]
train_cost: 0.049073067
valid_cost: 0.049914464
Accuracy: 0.9852
________________________________________
step: 0 - epoch: 2
y_hat: [7 2 1 0 4]
y    : [7 2 1 0 4]
train_cost: 0.039295744
valid_cost: 0.038178977
Accuracy: 0.9904
________________________________________
step: 0 - epoch: 3
y_hat: [7 2 1 0 4]
y    : [7 2 1 0 4]
train_cost: 0.03736692
valid_cost: 0.039528683
Accuracy: 0.9892
________________________________________
step: 0 - epoch: 4
y_hat: [7 2 1 0 4]
y    : [7 2 1 0 4]
train_cost: 0.016135832
valid_cost: 0.03878895
Accuracy: 0.9894
________________________________________


Optimization Finished!
Test accuracy: 0.9904
